In [ ]:
#default_exp loss

In [ ]:
#export
from fastai2.basics import *
from faststyle import *

In [ ]:
#export
def gram(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)

In [ ]:
#export
#TODO: hardcoded layers
# def get_stl_fs(fs): return fs[:-1]
# def get_cnt_fs(fs): return fs[-1:]

In [ ]:
#export
def StyleLoss(ws=None):
  def _inner(fs, y_fs, sims_fs):
    assert len(fs)==len(y_fs)==len(sims_fs)
    bs = fs[0].shape[0]
#     sims_gs = get_stl_fs([torch.stack([gram(x) for x in stl_t]).mean(axis=0) for stl_t in zip(*sims_fs)])
    sims_gs = L(torch.stack([gram(f) for f in fs]).wmean(ws, dim=0) for fs in sims_fs)
    pred_gs = [gram(f) for f in fs]
    assert len(sims_gs) == len(pred_gs)
    stl_losses = [F.mse_loss(g1.repeat(bs,1,1),g2) for g1,g2 in zip(sims_gs,pred_gs)]
    return sum(stl_losses)
  return _inner

In [ ]:
#export
def ContentLoss():
  def _inner(fs, y_fs, sims_fs):
    return sum([F.mse_loss(*o) for o in zip(fs,y_fs)])
  return _inner

In [ ]:
#export
def TVLoss():      
  def _inner(img):
    tv_h = ((img[:,:,1:,:] - img[:,:,:-1,:]).pow(2)).mean()
    tv_w = ((img[:,:,:,1:] - img[:,:,:,:-1]).pow(2)).mean()    
    return (tv_h + tv_w)
  return _inner

In [ ]:
#export
#TODO: Use @func_kwargs for losses
class FastStyleLoss2(Module):
  def __init__(self, get_fs, stl_w=3e5, cnt_w=1, tv_w=20):
    self.stl_loss_fn = StyleLoss()
    self.cnt_loss_fn = ContentLoss()
    self.tv_loss_fn  = TVLoss()
    store_attr(self, 'get_fs,stl_w,cnt_w,tv_w')
    self.metric_names = ['stl', 'cnt', 'tv']

  def forward(self, pred, targ, sims_fs, **kwargs):
    pred_feats,targ_feats = self.get_fs(pred),self.get_fs(targ)
#     assert not (pred_feats[0] == targ_feats[0]).all()
    self.stl = self.stl_w*self.stl_loss_fn(*L(pred_feats,targ_feats,sims_fs).attrgot('style'))
    self.cnt = self.cnt_w*self.cnt_loss_fn(*L(pred_feats,targ_feats,sims_fs).attrgot('content'))
    self.tv  = self.tv_w *self.tv_loss_fn(pred)
    return self.stl+self.cnt+self.tv

## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.ipynb.
Converted 02_layer_features.ipynb.
Converted 03_loss.ipynb.
Converted 04_learner.ipynb.
Converted 04_models.ipynb.
Converted 06_callback.ipynb.
